# Подготовка данных

## Для обучения модели были использованные данные из https://huggingface.co/datasets/rcp-meetings/rudialogsum_v2

In [ ]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
!pip3 install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("rcp-meetings/rudialogsum_v2")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog', 'summary'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['dialog', 'summary'],
        num_rows: 1500
    })
})

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df

,dialog,summary
0,"<speaker_1> Здравствуйте, мистер Смит. Я докто...","Мистер Смит проходит обследование, и доктор Хо..."
1,"<speaker_1> Здравствуйте, миссис Паркер, как у...",Миссис Паркер берет Рики на прививки. Доктор П...
2,"<speaker_1> Извините, вы не видели связку ключ...",<speaker_1> ищет набор ключей и просит <speake...
3,"<speaker_1> Почему ты не сказал мне, что у теб...","<speaker_1> злится, потому что <speaker_2> не ..."
4,"<speaker_1> Ватсап, дамы! Сегодня вечером ты б...",Малик приглашает Никки на танец. Никки соглаша...
...,...,...
12455,<speaker_1> Извините меня. Вы мистер Грин из М...,"Тан Линг забирает мистера Грина, которого легк..."
12456,"<speaker_1> Мистер Юинг сказал, что мы должны ...",<speaker_1> и <speaker_2> планируют вместе дое...
12457,<speaker_1> Чем я могу вам помочь сегодня?\n<s...,<speaker_2> арендует небольшой автомобиль на 5...
12458,<speaker_1> Ты сегодня выглядишь немного несча...,мама <speaker_2> потеряла работу. <speaker_2> ...


## Возьмем стобец с диалогами. Заменим токены на нужные (для модели tinkoff-ai/ruDialoGPT-medium)

In [ ]:
df_clear = df['dialog'].map(lambda x: x.replace("<speaker_1>", "@@ПЕРВЫЙ@@").replace("<speaker_2>", "@@ВТОРОЙ@@"))

In [ ]:
df_clear[2000]

'@@ПЕРВЫЙ@@ Папа, мне нужны кое-какие принадлежности для школы, и я хотел спросить, не нужно ли...\n@@ВТОРОЙ@@ Да. По-моему, в кухонном ящике есть пара карандашей и ластик.\n@@ПЕРВЫЙ@@ Папа, я сейчас учусь в 8 классе, и мне нужны настоящие принадлежности для моих сложных занятий.\n@@ВТОРОЙ@@ О, так тебе тоже нужна линейка?\n@@ПЕРВЫЙ@@ Папа, мне нужны какие-нибудь высокотехнологичные штуки вроде калькулятора и портативного компьютера.\n@@ВТОРОЙ@@ Эээ. У меня не было ничего подобного, когда я учился в средней школе, и я справлялся просто отлично.\n@@ПЕРВЫЙ@@ Да, и \nмашин тоже не было. Сейчас все намного прогрессивнее.\n@@ВТОРОЙ@@ Отлично. Моя дочь будет играть в видеоигры на уроке.\n@@ПЕРВЫЙ@@ Папа.\n@@ВТОРОЙ@@ Хорошо. Во сколько мне все это обойдется?\n@@ПЕРВЫЙ@@ Ну, я нашел компьютер в магазине всего за 299 долларов.\n@@ВТОРОЙ@@ Что ж, ты будешь поддерживать меня в старости, так что, думаю, да. Когда вам это нужно?\n@@ПЕРВЫЙ@@ Сейчас, прямо сейчас. Мама уже ждет нас в машине. Она сказ

In [ ]:
df_clear.to_csv("out.csv")

# Обучение модели. В качестве базовой модели была использована модель из https://huggingface.co/tinkoff-ai/ruDialoGPT-medium

In [ ]:
import csv

with open('/content/out.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    data = [row for row in reader]

# Преобразуем данные в текстовый файл
count = 0
with open('train.txt', 'w', encoding='utf-8') as f:
    for row in data:
        response = row['dialog']
        f.write(response)
        count += 1
        # ограничение для ускорения обучения
        if count == 300:
          break

In [ ]:
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Загрузка предобученной модели GPT
model_name = "tinkoff-ai/ruDialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
question = 'Привет друг!'
question_ids = tokenizer(question + tokenizer.eos_token, return_tensors='pt')

print(tokenizer.eos_token_id, tokenizer.bos_token_id, tokenizer.pad_token_id)
# 2, 1, 32000

print(question_ids)
# tensor([[    1, 15043, 29892,   727, 29991,   829, 29879, 29958]],

print(tokenizer.decode(tokenizer.pad_token_id))

50261 50261 0
{'input_ids': tensor([[37954,   702,     5, 50261]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
<pad>


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Загрузка предобученной модели GPT
model_name = "tinkoff-ai/ruDialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True)
model = AutoModelWithLMHead.from_pretrained(model_name)

# Подготовка данных для обучения
train_file = "/content/train.txt"
dataset = TextDataset(tokenizer=tokenizer, file_path=train_file, block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=1000,
    save_total_limit=2
)

# Создание экземпляра класса Trainer и запуск обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)
trainer.train()

In [ ]:
save_directory = './pt_save_pretrained'
#tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('/content/pt_save_pretrained')
inputs = tokenizer('@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ Расскажи, как ты пришла к такому? @@ВТОРОЙ@@ Я всегда это дела @@ПЕРВЫЙ@@ С какой целью? @@ВТОРОЙ@@', return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=10,
    top_p=0.95,
    num_beams=3,
    num_return_sequences=3,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.2,
    repetition_penalty=1.2,
    length_penalty=1.0,
    eos_token_id=50257,
    max_new_tokens=40
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response

## Публикация модели для дальнейшего использования в телеграм боте (API)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("MaximMS/myDialogModel", create_pr=1)

In [ ]:
tokenizer.push_to_hub("MaximMS/myDialogModel", create_pr=1)